In [1]:
from stock_bot.TradingSystem import *
from stock_bot.AlpacaDataManager import *
from stock_bot.DataFrameBacktest import *
from stock_bot.TA_LIB_FunctionMapping import *
from pathlib import Path
from matplotlib.pyplot import hist


In [2]:


data_folder = Path('.\\config')
yaml_file = 'api_keys.yaml'
strategy  = 'ta-lib_example.yaml'
yaml_path = data_folder / yaml_file
strat_file = data_folder / strategy

with open(yaml_path, 'r') as file:
    yaml_config = yaml.safe_load(file)

api_key=yaml_config['api_key_paper']
api_secret=yaml_config['api_secret_paper']
data_fetcher = AlpacaDataFetcher(api_key, api_secret)

# Load your strategy
#strategy = Strategy(strat_file)

# Fetch historical data
historical_data = data_fetcher.get_historical_data(
    symbol="PLTR",
    timeframe="1m",
    start_date=datetime(2024, 1, 1, tzinfo=pytz.UTC),
    end_date=datetime(2024, 6, 20, tzinfo=pytz.UTC)
)

historical_data = historical_data.droplevel('symbol')
historical_data.tail()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2024-06-13 17:06:00+00:00,23.385,23.4000,23.3300,23.3450,375755.0,2179.0,23.374096
2024-06-13 17:17:00+00:00,23.348,23.3900,23.3000,23.3663,320483.0,1920.0,23.340907
2024-06-13 17:28:00+00:00,23.370,23.3899,23.2800,23.3050,295588.0,1972.0,23.327190
2024-06-13 17:39:00+00:00,23.315,23.3600,23.3143,23.3350,244751.0,1671.0,23.333286
2024-06-13 17:50:00+00:00,23.335,23.4200,23.3301,23.4100,408112.0,2332.0,23.382471


In [3]:
with open("C:\\Users\\fwmac\\Documents\\Projects\\alfraca\\true-bautist\\config\\ta-lib_example.yaml", 'r') as file:
    yaml_trade_config = yaml.safe_load(file)

In [4]:
yaml_trade_config

{'symbol': 'ETHUSD',
 'timeframe': '1d',
 'mode': 'backtest',
 'risk_management': {'position_sizing_method': 'atr_based',
  'risk_per_trade': 0.05,
  'max_position_size': 1000,
  'stop_loss': 0.02,
  'take_profit': 0.1,
  'max_drawdown': 0.15,
  'atr_multiplier': 2.0},
 'indicators': [{'name': 'RSI', 'params': {'timeperiod': 14}},
  {'name': 'BBANDS',
   'params': {'timeperiod': 18, 'nbdevup': 2, 'nbdevdn': 2, 'matype': 0}},
  {'name': 'MACD',
   'params': {'fastperiod': 12, 'slowperiod': 26, 'signalperiod': 9}},
  {'name': 'ATR', 'params': {'timeperiod': 14}}],
 'entry_conditions': [{'indicator': 'MACD',
   'comparison': 'crosses_above',
   'value': 'signal'},
  {'indicator': 'BBANDS', 'comparison': 'crosses_below', 'value': 'lower'}],
 'exit_conditions': [{'indicator': 'RSI', 'comparison': 'above', 'value': 70},
  {'indicator': 'ATR', 'comparison': 'above', 'value': 25}]}

In [5]:
# Initialize indicators
indicators = TALibIndicators()

# Calculate indicators for your DataFrame
def calculate_indicators(df: pd.DataFrame, config: Dict) -> pd.DataFrame:
    result_df = df.copy()
    
    for ind_config in config['indicators']:
        name = ind_config['name']
        params = ind_config.get('params', {})
        
        # Calculate indicator
        indicator_df = indicators.calculate_indicator(name, df, params)

        # Merge results
        result_df = pd.concat([result_df, indicator_df], axis=1)
    
    return result_df

# Use in your backtest
df = calculate_indicators(historical_data, yaml_trade_config)

In [6]:
df

,open,high,low,close,volume,trade_count,vwap,rsi,upperband,middleband,lowerband,macd,macdsignal,macdhist,atr
timestamp,,,,,,,,,,,,,,,
2024-01-02 08:53:00+00:00,17.460,17.4600,17.2100,17.3300,29837.0,206.0,17.344132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:04:00+00:00,17.330,17.3300,17.2700,17.2800,9630.0,142.0,17.285152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:15:00+00:00,17.260,17.2700,17.2500,17.2700,10839.0,117.0,17.261493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:26:00+00:00,17.270,17.3000,17.2600,17.3000,2580.0,36.0,17.288342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:37:00+00:00,17.290,17.3100,17.2700,17.3100,16999.0,161.0,17.290022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13 17:06:00+00:00,23.385,23.4000,23.3300,23.3450,375755.0,2179.0,23.374096,40.153360,23.601378,23.360194,23.119011,-0.117726,-0.126318,0.008591,0.098721
2024-06-13 17:17:00+00:00,23.348,23.3900,23.3000,23.3663,320483.0,1920.0,23.340907,42.241651,23.510238,23.340822,23.171406,-0.106607,-0.122375,0.015768,0.098098
2024-06-13 17:28:00+00:00,23.370,23.3899,23.2800,23.3050,295588.0,1972.0,23.327190,38.119156,23.496324,23.333317,23.170309,-0.101571,-0.118214,0.016644,0.098941


In [7]:
for col in df.columns:
    if col not in ['open', 'high', 'low', 'volume']:  # Skip some columns
        df[f'{col}_prev'] = df[col].shift(1)

In [8]:
df  

,open,high,low,close,volume,trade_count,vwap,rsi,upperband,middleband,...,trade_count_prev,vwap_prev,rsi_prev,upperband_prev,middleband_prev,lowerband_prev,macd_prev,macdsignal_prev,macdhist_prev,atr_prev
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 08:53:00+00:00,17.460,17.4600,17.2100,17.3300,29837.0,206.0,17.344132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:04:00+00:00,17.330,17.3300,17.2700,17.2800,9630.0,142.0,17.285152,NaN,NaN,NaN,...,206.0,17.344132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:15:00+00:00,17.260,17.2700,17.2500,17.2700,10839.0,117.0,17.261493,NaN,NaN,NaN,...,142.0,17.285152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:26:00+00:00,17.270,17.3000,17.2600,17.3000,2580.0,36.0,17.288342,NaN,NaN,NaN,...,117.0,17.261493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02 09:37:00+00:00,17.290,17.3100,17.2700,17.3100,16999.0,161.0,17.290022,NaN,NaN,NaN,...,36.0,17.288342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13 17:06:00+00:00,23.385,23.4000,23.3300,23.3450,375755.0,2179.0,23.374096,40.153360,23.601378,23.360194,...,2714.0,23.301227,42.855305,23.671272,23.380750,23.090228,-0.128037,-0.128465,0.000428,0.100931
2024-06-13 17:17:00+00:00,23.348,23.3900,23.3000,23.3663,320483.0,1920.0,23.340907,42.241651,23.510238,23.340822,...,2179.0,23.374096,40.153360,23.601378,23.360194,23.119011,-0.117726,-0.126318,0.008591,0.098721
2024-06-13 17:28:00+00:00,23.370,23.3899,23.2800,23.3050,295588.0,1972.0,23.327190,38.119156,23.496324,23.333317,...,1920.0,23.340907,42.241651,23.510238,23.340822,23.171406,-0.106607,-0.122375,0.015768,0.098098


In [9]:
class Condition(ABC):
    def __init__(self, indicator: str, comparison: str, value: Any):
        self.indicator = indicator
        self.comparison = comparison
        self.value = value
        

    @abstractmethod
    def evaluate(self, row: pd.Series) -> bool:
        pass

class IndicatorCondition(Condition):
    def __str__(self):
        return f'The indicator type is {self.indicator} and the comparison is {self.comparison} using {self.value}'

    def __repr__(self):
        return f'IndicatorCondition(\'{self.indicator}\', {self.comparison}, {self.value})'
    
    def evaluate(self, row: pd.Series) -> bool:
        # Print available columns for debugging
        print(f"\nEvaluating {self.indicator} condition:")
        print(f"Comparison: {self.comparison}")
        print(f"Value: {self.value}")
        
        # Handle special cases for indicators
        if self.indicator == "MACD" and self.value == "signal":
            macd_val = row.get('macd')
            signal_val = row.get('macdsignal')
            macd_prev = row.get('macd_prev')
            signal_prev = row.get('macdsignal_prev')
            
            print(f"MACD values:")
            print(f"Current MACD: {macd_val}")
            print(f"Current Signal: {signal_val}")
            print(f"Previous MACD: {macd_prev}")
            print(f"Previous Signal: {signal_prev}")
            
            if any(pd.isna([macd_val, signal_val, macd_prev, signal_prev])):
                print("Missing or NaN values detected")
                return False
                
            if self.comparison == "crosses_above":
                result = (macd_prev <= signal_prev) and (macd_val > signal_val)
                print(f"Crosses above evaluation: {result}")
                return result
            elif self.comparison == "crosses_below":
                result = (macd_prev >= signal_prev) and (macd_val < signal_val)
                print(f"Crosses below evaluation: {result}")
                return result
                
        elif self.indicator == "BBANDS" and self.value == "lower":
            price = row.get('close')
            lower = row.get('lowerband')  # lower band
            price_prev = row.get('close_prev')
            lower_prev = row.get('lowerband_prev')
            
            print(f"\nBBands check:")
            print(f"Current price: {price}")
            print(f"Current lower band: {lower}")
            print(f"Previous price: {price_prev}")
            print(f"Previous lower band: {lower_prev}")
            
            if any(pd.isna([price, lower, price_prev, lower_prev])):
                print("Missing or NaN values detected")
                return False
                
            if self.comparison == "crosses_below":
                result = price_prev >= lower_prev and price < lower
                print(f"Crosses below evaluation: {result}")
                return result
            elif self.comparison == "crosses_above":
                result = price_prev <= lower_prev and price > lower
                print(f"Crosses above evaluation: {result}")
                return result
                
        else:
            # Standard indicator comparison
            indicator_value = row.get(self.indicator.lower())
            if pd.isna(indicator_value):
                return False

            if isinstance(self.value, (int, float)):
                compare_value = self.value
            else:
                compare_value = row.get(self.value.lower())
                if pd.isna(compare_value):
                    return False

            if self.comparison == "above":
                print("indicator value: ", indicator_value, " / compare value: ", compare_value)
                return indicator_value > compare_value
            elif self.comparison == "below":
                print("indicator value: ", indicator_value, " / compare value: ", compare_value)
                return indicator_value < compare_value

        return False

In [10]:
def _setup_conditions(conditions_config: List[Dict]) -> List[Condition]:
    return [IndicatorCondition(
        indicator=cond['indicator'],
        comparison=cond['comparison'],
        value=cond['value']
    ) for cond in conditions_config]

def _check_conditions(conditions: List[Condition], row: pd.Series) -> bool:
    """Evaluate all conditions together and return True only if all are met"""
    print("\nChecking all conditions:")
    results = []
    for condition in conditions:
        result = condition.evaluate(row)
        results.append(result)
        print(f"- {condition.indicator} {condition.comparison} {condition.value}: {result}")
    
    all_conditions_met = all(results)
    print(f"All conditions met: {all_conditions_met}")
    return all_conditions_met


'''
def _check_entry_conditions(row: pd.Series) -> bool:
    return _check_conditions(entry_conditions, row)

def _check_exit_conditions(row: pd.Series) -> bool:
    return _check_conditions(exit_conditions, row)'''

'\ndef _check_entry_conditions(row: pd.Series) -> bool:\n    return _check_conditions(entry_conditions, row)\n\ndef _check_exit_conditions(row: pd.Series) -> bool:\n    return _check_conditions(exit_conditions, row)'

In [11]:
entry_conditions = _setup_conditions(yaml_trade_config.get('entry_conditions'))
print(str(entry_conditions[1]))

The indicator type is BBANDS and the comparison is crosses_below using lower


In [12]:
entry_conditions[0].evaluate(df.iloc[1000])


Evaluating MACD condition:
Comparison: crosses_above
Value: signal
MACD values:
Current MACD: -0.016223170093958572
Current Signal: 0.04012665826813058
Previous MACD: 0.015609030987167927
Previous Signal: 0.05421411535865287
Crosses above evaluation: False


np.False_

In [13]:

buys = 0
len_entry_conditions = len(entry_conditions)
for i, (idx, row) in enumerate(df.iterrows()):
 
    for cond in entry_conditions:
        res = cond.evaluate(row)
        print(i, res)

    if len_entry_conditions == num_conditions_met:
        print("signal buy")
        buys += 1
        


Evaluating MACD condition:
Comparison: crosses_above
Value: signal
MACD values:
Current MACD: nan
Current Signal: nan
Previous MACD: nan
Previous Signal: nan
Missing or NaN values detected
0 False

Evaluating BBANDS condition:
Comparison: crosses_below
Value: lower

BBands check:
Current price: 17.33
Current lower band: nan
Previous price: nan
Previous lower band: nan
Missing or NaN values detected
0 False


NameError: name 'num_conditions_met' is not defined

In [ ]:
entry_conditions[1].evaluate(df.iloc[1000])

In [ ]:
exit_conditions = _setup_conditions(yaml_trade_config.get('exit_conditions'))
exit_conditions[0].evaluate(df.iloc[1000])

In [ ]:
exit_conditions[1].evaluate(df.iloc[1000])

In [19]:
dataframe_backtester = DataFrameBacktester(strat_file)

In [ ]:
results = dataframe_backtester.backtest(historical_data)

In [ ]:
results.evaluation_df

In [ ]:
results.trades